# **Template OP on salish**

In [58]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, AdvectionRK4, AdvectionRK4_3D

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels import DeleteParticle, Buoyancy
from OP_functions import *

In [59]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

## Definitions

In [60]:
start = datetime(2018, 12, 23) #Start date
# Set Time length [days] and timestep [seconds]
length = 15 
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 6 # number of deploying locations
n = 100 # 1000   # number of particles per location
dmin = [0,0,0,0,0,70] #minimum depth
dd = 20 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [61]:
from parcels import Variable

class PEParticle(JITParticle):         # Define a new particle class
    ro = Variable('ro', initial = 1370)             # desnsity PE
    size = Variable('size', initial = 5e-5)

In [62]:
Dat=xr.open_dataset(paths['coords'],decode_times=False)
outf_lat=Dat['nav_lat'][445,304]
outf_lon=Dat['nav_lon'][445,304]

In [63]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
# Execute run
clon, clat = [-123.901172,-125.155849,-123.207648,-122.427508,-123.399769,float(outf_lon)], [49.186308,49.975326,49.305448,47.622403,48.399420,float(outf_lat)]  # choose horizontal centre of the particle cloud
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [64]:
#Set start date time and the name of the output file
name = 'PETStokes' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/analysis-jose/notebooks/results/PETStokes20181223_1n_20190107_1n.nc


## Simulation

In [65]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

In [66]:
# #Load Salish output as fields
field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, PEParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

p_flt = pset.Kernel(Buoyancy) #uncomment if you want to use additional kernels, also change the value of pset.Kernel to "AdvectionRK4_3D + p_flt"
##pset.computeTimeChunk(allow_time_extrapolation=1)
pset.execute(AdvectionRK4_3D + p_flt, 
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

Exception ignored in: <function KernelSOA.__del__ at 0x7f0fc8887dc0>
Traceback (most recent call last):
  File "/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/parcels/kernel/kernelsoa.py", line 157, in __del__
    super(KernelSOA, self).__del__()
  File "/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/parcels/kernel/basekernel.py", line 99, in __del__
    self.remove_lib()
  File "/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/parcels/kernel/basekernel.py", line 185, in remove_lib
    if self.src_file is None:
AttributeError: 'KernelSOA' object has no attribute 'src_file'
Exception ignored in: <function KernelSOA.__del__ at 0x7f0fc8887dc0>
Traceback (most recent call last):
  File "/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/parcels/kernel/kernelsoa.py", line 157, in __del__
    super(KernelSOA, self).__del__()
  File "/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/parcels/kernel/basekernel.py", line 99, in __